In [20]:
from lxml import html
import requests
import re
import csv
import time 
import pandas as pd 
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup as bs
from selenium import webdriver 
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from webdriver_manager.chrome import ChromeDriverManager

def html_parse_tree(url):
    page = requests.get(url)
    tree = html.fromstring(page.content)
    return tree

def xpath_parse(tree, xpath):
    result = tree.xpath(xpath)
    return result

def regex_strip_array(array):
    for i in range(0, len(array)):
        array[i] = regex_strip_string(array[i]).strip()
    return array

def regex_strip_string(string):
    string = re.sub('\n', '', string).strip()
    string = re.sub('\r', '', string).strip()
    string = re.sub('\t', '', string).strip()
    return string

def format_spacing(max_spacing, variable):
    spacing_count = max_spacing - len(variable)
    output = ''
    for i in range(0, spacing_count):
        output += ' '
    return output

def fraction_stats(string):
    string = string.replace('(', '')
    string = string.replace(')', '')
    return string.split('/')

def add2csv(array, filename):
    with open(filename, 'a') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(array)

def array2csv(array, filename):
    with open(filename, "w+") as my_csv:
        csvWriter = csv.writer(my_csv, delimiter = ',')
        csvWriter.writerows(array)

In [23]:
year_url = "http://www.atptour.com/en/scores/results-archive?year=2022"
url_prefix = "http://www.atptour.com"

# HTML tree

options = ChromeOptions()
options.add_argument("--headless=new")
options.page_load_strategy = 'normal' 
chrome_path = "./bin/chromedriver.exe" #ChromeDriverManager()#.install() 
chrome_service = Service(chrome_path) 
driver = Chrome(options=options, service=chrome_service) 
driver.implicitly_wait(5)
#atpurl = "https://www.atptour.com/en/scores/current/us-open/560/daily-schedule"
driver.get(year_url) 


year_tree = html.fromstring(driver.page_source)  #html_parse_tree(year_url)

    # Initial XPath to find number of tournaments in a given year
tourney_titles_xpath = "//span[contains(@class, 'tourney-title')]/text()"
tourney_titles_parsed = xpath_parse(year_tree, tourney_titles_xpath)
tourney_titles_cleaned = regex_strip_array(tourney_titles_parsed)

# If no tournaments found in <span> tags try find in <a> tags
if len(tourney_titles_cleaned) == 0:
    tourney_titles_xpath = "//a[contains(@class, 'tourney-title')]/text()"
    tourney_titles_parsed = xpath_parse(year_tree, tourney_titles_xpath)
    tourney_titles_cleaned = regex_strip_array(tourney_titles_parsed)

print(tourney_titles_cleaned)

tourney_count = len(tourney_titles_cleaned)


['ATP Cup', 'Adelaide 1', 'Melbourne', 'Sydney', 'Adelaide 2', 'Australian Open', 'Cordoba', 'Pune', 'Montpellier', 'Rotterdam', 'Dallas', 'Buenos Aires', 'Rio de Janeiro', 'Doha', 'Delray Beach', 'Marseille', 'Dubai', 'Acapulco', 'Santiago', 'ATP Masters 1000 Indian Wells', 'ATP Masters 1000 Miami', 'Houston', 'Marrakech', 'ATP Masters 1000 Monte Carlo', 'Barcelona', 'Belgrade', 'Estoril', 'Munich', 'ATP Masters 1000 Madrid', 'ATP Masters 1000 Rome', 'Geneva', 'Lyon', 'Roland Garros', 'Stuttgart', "'s-Hertogenbosch", 'Halle', "London / Queen's Club", 'Mallorca', 'Eastbourne', 'Wimbledon', 'Newport', 'Bastad', 'Hamburg', 'Gstaad', 'Atlanta', 'Kitzbuhel', 'Umag', 'Washington', 'Los Cabos', 'ATP Masters 1000 Canada', 'ATP Masters 1000 Cincinnati', 'Winston-Salem', 'US Open', 'San Diego', 'Metz', 'Laver Cup', 'Seoul', 'Tel Aviv', 'Sofia', 'Tokyo', 'Astana', 'Gijon', 'Florence', 'Antwerp', 'Stockholm', 'Naples', 'Vienna', 'Basel', 'ATP Masters 1000 Paris', 'Intesa Sanpaolo Next Gen ATP Fin

In [20]:
import pandas as pd 
matchs = pd.read_csv("./data/matchs.csv")


for index, match in matchs.iterrows():
    print(match.match_stats_url_suffix)

/en/scores/stats-centre/archive/2023/2843/ms001
/en/scores/stats-centre/archive/2023/2843/ms002
/en/scores/stats-centre/archive/2023/2843/ms003
/en/scores/stats-centre/archive/2023/2843/ms004
/en/scores/stats-centre/archive/2023/2843/ms005
/en/scores/stats-centre/archive/2023/2843/ms006
/en/scores/stats-centre/archive/2023/2843/ms007
/en/scores/stats-centre/archive/2023/2843/ms008
/en/scores/stats-centre/archive/2023/2843/ms010
/en/scores/stats-centre/archive/2023/2843/ms012
/en/scores/stats-centre/archive/2023/2843/ms009
/en/scores/stats-centre/archive/2023/2843/ms011
/en/scores/stats-centre/archive/2023/2843/ms013
/en/scores/stats-centre/archive/2023/2843/ms014
/en/scores/stats-centre/archive/2023/2843/ms015
/en/scores/stats-centre/archive/2023/2843/ms016
/en/scores/stats-centre/archive/2023/2843/ms031
/en/scores/stats-centre/archive/2023/2843/ms020
/en/scores/stats-centre/archive/2023/2843/ms027
/en/scores/stats-centre/archive/2023/2843/ms028
/en/scores/stats-centre/archive/2023/284